In [1]:
using RigidBodyDynamics, RigidBodyDynamics.Contact, RigidBodyDynamics.OdeIntegrators
using MomentumBasedControl
using MixedIntegerExperiments
using StaticArrays
using RigidBodyTreeInspector, DrakeVisualizer
using ValkyrieRobot
using BipedControlUtil

In [2]:
val = Valkyrie()
mechanism = val.mechanism;

In [3]:
# add environment
rootframe = root_frame(mechanism)
floor = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
leftwall = HalfSpace3D(Point3D(rootframe, 0., -1., 1.5), FreeVector3D(rootframe, 0., 1., 0.))
rightwall = HalfSpace3D(Point3D(rootframe, 0, 1., 1.5), FreeVector3D(rootframe, 0., -1., 0.))
add_environment_primitive!(mechanism, floor)
# add_environment_primitive!(mechanism, leftwall)
# add_environment_primitive!(mechanism, rightwall)
;

In [4]:
# Create visualizer
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1))
vis = Visualizer(parse_urdf(ValkyrieRobot.urdfpath(), mechanism; package_path = [ValkyrieRobot.packagepath()]), :valkyrie);
visualize_environment(mechanism, vis)

In [5]:
remove_fixed_tree_joints!(mechanism)
const state = MechanismState(Float64, mechanism)
const controllerstate = MomentumBasedControllerState(state)
;

In [6]:
# Initial setup
zero!(state)
kneebend = 1.
for side in instances(Side)
    set_configuration!(state, val.knees[side], [kneebend])
    set_configuration!(state, val.hippitches[side], [-kneebend / 2])
    set_configuration!(state, val.anklepitches[side], [-kneebend / 2])
end
set_configuration!(state, val.floatingjoint, [1; 0; 0; 0; 0; 0; 1.07]) # TODO
settransform!(vis, state)
MomentumBasedControl.reset!(controllerstate)
;

In [7]:
const τ = Vector{Float64}(num_velocities(mechanism))
const result = DynamicsResult(Float64, mechanism)
function damped_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    τ[:] = velocity(state)
    scale!(τ, -1.0)
    dynamics!(result, state, τ)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

damped_dynamics! (generic function with 1 method)

In [8]:
const controller = MomentumBasedController{Float64}(mechanism, 5e-3);
contacts = add_mechanism_contacts!(controller)
jointacceltasks = add_mechanism_joint_accel_tasks!(controller);

In [9]:
comgains = PDGains(10., 2.)

MomentumBasedControl.PDGains{Float64}(10.0,2.0)

In [10]:
# Foot accelerations w.r.t. world
footacceltasks = Dict{Side, SpatialAccelerationTask{Float64}}()
legjoints = []
for side in instances(Side)
    foot = val.feet[side]
    world_to_foot = path(mechanism, root_body(mechanism), foot)
    pelvis_to_foot = path(mechanism, val.pelvis, foot)
    append!(legjoints, joint for (joint, direction) in pelvis_to_foot)
    frame = default_frame(foot)
    task = footacceltasks[side] = SpatialAccelerationTask(world_to_foot, frame, eye(3, 3), eye(3, 3))
    add!(controller, task)
end

In [11]:
# Pelvis acceleration w.r.t. world
pelvisacceltask = SpatialAccelerationTask(path(mechanism, root_body(mechanism), val.pelvis), default_frame(val.pelvis), eye(3, 3), zeros(0, 3))
add!(controller, pelvisacceltask)
pelvisgains = PDGains(10., 2.)
;

In [12]:
# Position control tasks
# revolutejoints = filter(j -> isa(j.jointType, Revolute), joints(mechanism))
# positioncontroltasks = Dict(joint => JointAccelerationTask(joint) for joint in setdiff(revolutejoints, legjoints))
# foreach(task -> add!(controller, task), values(positioncontroltasks))
# positioncontrolgains = Dict(joint => PDGains(10.0, 2.) for joint in keys(positioncontroltasks))
# ;

In [13]:
const result = DynamicsResult(Float64, mechanism)
feet = val.feet
com0 = center_of_mass(state)
com0 += FreeVector3D(com0.frame, 0.0, 0.0, -0.05)
μ = 0.8
normal = floor.outward_normal
function controlled_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    # TODO: extract out: highlevelcontrol
    # Foot accelerations
    T = eltype(controller)
    MomentumBasedControl.reset!(controller)
    regularize_joint_accels!(controller, 0.05)
    for side in instances(Side)
        task = footacceltasks[side]
        base = RigidBodyDynamics.Graphs.source(task.path)
        body = RigidBodyDynamics.Graphs.target(task.path)
        accel = zero(SpatialAcceleration{T}, default_frame(body), default_frame(base), default_frame(body))
        set!(task, accel, Inf)
        
        for contactsettings in contacts[body]
            set!(contactsettings, 1e-1, μ, transform(state, normal, default_frame(body)))
        end
    end
    
    # Pelvis orientation control
    Hpelvis = transform_to_root(state, val.pelvis)
    Tpelvis = transform(twist_wrt_world(state, val.pelvis), inv(Hpelvis))
    ωdpelvis = pd(pelvisgains, rotation(Hpelvis), Tpelvis.angular)
    Ṫpelvis = SpatialAcceleration(Tpelvis.body, Tpelvis.base, Tpelvis.frame, ωdpelvis, zero(ωdpelvis))
    set!(pelvisacceltask, Ṫpelvis, Inf)
    
#     # Joint position control
#     for (joint, task) in positioncontroltasks
#         gains = positioncontrolgains[joint]
#         set!(task, pd(gains, configuration(state, joint), 0., velocity(state, joint), 0.), Inf)
#     end
    
    # Centroidal momentum control
    com = center_of_mass(state)
    centroidal = centroidal_frame(controller)
    world_to_centroidal = Transform3D(com.frame, centroidal, -com.v)
    h = transform(momentum(state), world_to_centroidal)
    comerror = FreeVector3D(transform(com0, world_to_centroidal))
    comderror = -FreeVector3D(h.frame, h.linear)
    ldot_desired = comgains.k * comerror + comgains.d * comderror
    hdot_desired = Wrench(ldot_desired, zero(ldot_desired))
    add!(controller, MomentumRateTask(hdot_desired, eye(SMatrix{3, 3}), eye(SMatrix{3, 3}), 0.))
    
    τ = control(controller, t, controllerstate)
    dynamics!(result, state, τ)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

controlled_dynamics! (generic function with 1 method)

In [14]:
# Simulation
const dyn! = controlled_dynamics!
# const dyn! = damped_dynamics!
const integrator = MuntheKaasIntegrator(dyn!, runge_kutta_4(Float64), DrakeVisualizerSink(vis));

In [15]:
integrate(integrator, state, 1., 1e-4, maxRealtimeRate = Inf)

LoadError: Invalid coefficient NaN on variable v̇[1]